In [2]:
# Plant Identification & Care Suggestion Notebook
# --------------------------------------------------------------
# This script demonstrates a minimal-data transfer-learning pipeline
# to classify 10 plant species and map predictions to care tips using PlantNet300K pre-trained ResNet18.

# Install & import dependencies
!pip install torch torchvision

import os
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision.models import resnet18
from torch.utils.data import DataLoader




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [1]:
# Mount Drive (if storing data there)
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [4]:
# Extract uploaded dataset
import zipfile

zip_path = '/content/Finalized.zip'  # adjust if you upload to another location
extract_path = '/content/extracted_images'  # matches expected train_dir

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete. Images are now in:", extract_path)


Extraction complete. Images are now in: /content/extracted_images


In [5]:
# Define parameters
NUM_CLASSES = 10
IMAGE_SIZE = 224
BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 1e-4
USE_GPU = torch.cuda.is_available()



In [6]:
# Define transform pipeline
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])



In [7]:
# Load dataset
# Image structure: extracted_images/data/<species_name>/<image files>
train_dir = '/content/extracted_images/data'
train_dataset = datasets.ImageFolder(train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)



In [15]:
# Load pre-trained ResNet18 model
# Make sure that utils.py and the pre-trained model is uploaded
from utils import load_model
from torchvision.models import resnet18

filename = 'resnet18_weights_best_acc.tar' # pre-trained model path
use_gpu = True  # load weights on the gpu
model = resnet18(num_classes=1081) # 1081 classes in Pl@ntNet-300K

load_model(model, filename=filename, use_gpu=use_gpu)


29

In [16]:
# Replace final layer for fine-tuning on your 10-plant subset
model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
if USE_GPU:
    model = model.cuda()



In [17]:
# Define optimizer and loss
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)




In [18]:
# Train loop
model.train()
for epoch in range(EPOCHS):
    running_loss = 0.0
    for images, labels in train_loader:
        if USE_GPU:
            images, labels = images.cuda(), labels.cuda()
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {running_loss/len(train_loader):.4f}")



Epoch [1/10], Loss: 0.4534
Epoch [2/10], Loss: 0.1315
Epoch [3/10], Loss: 0.0779
Epoch [4/10], Loss: 0.0666
Epoch [5/10], Loss: 0.0471
Epoch [6/10], Loss: 0.0379
Epoch [7/10], Loss: 0.0387
Epoch [8/10], Loss: 0.0323
Epoch [9/10], Loss: 0.0262
Epoch [10/10], Loss: 0.0316


In [19]:
# Save model
torch.save(model.state_dict(), '/content/plantnet_finetuned_resnet18.pth')



In [29]:
# Class to care tips mapping
CARE_TIPS = {
    'Daucus_carota': 'Full sun; light, well-drained soil; keep consistently moist. Sow in spring or fall.',
    'Alliaria_petiolata': 'Tolerates shade; grows in disturbed soils; invasive—control spread.',
    'Hypericum_perforatum': 'Full sun; dry to medium soil; drought tolerant once established.',
    'Centranthus_ruber': 'Full sun; well-drained soil; cut back after flowering to promote rebloom.',
    'Cirsium_vulgare': 'Full sun; thrives in poor soils; invasive—control if not desired.',
    'Trifolium_pratense': 'Full sun to partial shade; moist, fertile soil; fixes nitrogen naturally.',
    'Calendula_officinalis': 'Full sun; moderate watering; deadhead for continuous bloom.',
    'Lamium_purpureum': 'Partial shade; moist, rich soil; often grows as a groundcover.',
    'Alcea_rosea': 'Full sun; well-drained soil; stake tall varieties for support.',
    'Papaver_rhoeas': 'Full sun; light soil; sow directly—does not transplant well.'
}

In [21]:
# Inference helper
from PIL import Image

def predict_and_suggest(img_path, model, class_map):
    model.eval()
    img = Image.open(img_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0)
    if USE_GPU:
        img_tensor = img_tensor.cuda()
    with torch.no_grad():
        outputs = model(img_tensor)
        _, predicted = torch.max(outputs, 1)
        class_idx = predicted.item()
    label = class_map[class_idx]
    tip = CARE_TIPS.get(label, 'No care tip available.')
    return label, tip



In [40]:
# Example usage:
# label, tip = predict_and_suggest('/content/data/test/myplant.jpg', model, train_dataset.classes)
# print(f"{label}: {tip}")

label, tip = predict_and_suggest('/content/myplant.jpg', model, train_dataset.classes)
print(f"{label}: {tip}")



Daucus_carota: Full sun; light, well-drained soil; keep consistently moist. Sow in spring or fall.
